In [ ]:
# folders
data_path = 'xxx' # todo
results_path = 'xxx' # todo

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms
import torch

# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))


data_dir = data_path + '/Data/MedMNIST/'
with np.load(data_dir+'organ'+'a'+'mnist.npz') as data:
    imgs = data['train_images']
    print(imgs.shape)
# with np.load(data_dir+'organ'+'c'+'mnist.npz') as data:
#     imgs = data['train_images']
#     print(imgs.shape)
# with np.load(data_dir+'organ'+'s'+'mnist.npz') as data:
#     imgs = data['train_images']
#     print(imgs.shape)

# get the image
curr_img = imgs[7688,:,:]
print(curr_img.max() != 0)
curr_img = curr_img*(255/curr_img.max())
curr_img = np.stack((curr_img,)*3, axis=-1)

img = Image.fromarray(curr_img.astype(np.uint8))
     
transformList = []
transformList.append(transforms.Resize(size=32))
transformList.append(transforms.ToTensor())
train_transform=transforms.Compose(transformList)
img = train_transform(img)   
plt.imshow(img.permute(1,2,0),cmap='gray')
plt.show()
print('End Size: '+ str(img.shape))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

datapoints_train = np.array([[1956,1153,1148],
         [1408, 626, 637],
         [1359, 608, 615],
         [1474, 600, 721],
         [3963,1088,1132],
         [3817,1170,1119],
         [6164,2986,3464],
         [3919,1002, 741],
         [3929,1022, 803],
         [3031,1173,2004],
         [3561,1572,1556]])
datapoints_val = np.array([[ 321, 191, 188],
         [ 233, 102, 104],
         [ 225,  96,  95],
         [ 392, 202, 246],
         [ 568, 132, 140],
         [ 637, 157, 159],
         [1033, 429, 491],
         [1033, 347, 261],
         [1009, 352, 275],
         [ 529, 179, 280],
         [ 511, 205, 213]])
datapoints_test = np.array([[1036, 833, 811],
 [ 784, 442, 439],
 [ 793, 441, 447],
 [ 785, 421, 510],
 [2064, 732, 704],
 [1965, 737, 693],
 [3285,1836,2078],
 [1747, 550, 397],
 [1813, 558, 439],
 [1622, 750,1343],
 [1884, 968, 968]])
print(datapoints_train.sum())
print(datapoints_val.sum())
print(datapoints_test.sum())

y_list =['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen']
x_list = ['Axial','Coronal','Sagittal']

# train
fig = plt.figure(figsize=(5, 5), dpi=80)
plt.imshow(datapoints_train) 
plt.xticks(np.arange(len(x_list)), labels=x_list, rotation=45)
plt.yticks(np.arange(len(y_list)), labels=y_list, rotation=0)
plt.colorbar()
# plt.show()
# plt.savefig('figs/medmnist_data_train.eps', bbox_inches='tight', format='eps')
plt.savefig('figs/medmnist_data_train.pdf', bbox_inches='tight', format='pdf')

# val
fig = plt.figure(figsize=(5, 5), dpi=80)
plt.imshow(datapoints_val) 
plt.xticks(np.arange(len(x_list)), labels=x_list, rotation=45)
plt.yticks(np.arange(len(y_list)), labels=y_list, rotation=0)
plt.colorbar()
# plt.show()
# plt.savefig('figs/medmnist_data_val.eps', bbox_inches='tight', format='eps')
plt.savefig('figs/medmnist_data_val.pdf', bbox_inches='tight', format='pdf')

# test
fig = plt.figure(figsize=(5, 5), dpi=80)
plt.imshow(datapoints_test) 
plt.xticks(np.arange(len(x_list)), labels=x_list, rotation=45)
plt.yticks(np.arange(len(y_list)), labels=y_list, rotation=0)
plt.colorbar()
# plt.show()
# plt.savefig('figs/medmnist_data_test.eps', bbox_inches='tight', format='eps')
plt.savefig('figs/medmnist_data_test.pdf', bbox_inches='tight', format='pdf')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms

data_dir = data_path + '/Data/MedMNIST/'

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 
    
# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

# load data
print('Annotations loading..')
imgs = np.empty((0,28,28))
label_vec = np.empty((0,2))
cnt = 0
curr_phase = 'train'
for i in ['a','c','s']:
    data_path = data_dir+'organ'+i+'mnist.npz'
    with np.load(data_path) as data:
        imgs = np.concatenate((imgs,data[curr_phase+'_images']),axis=0)
        labels = data[curr_phase+'_labels']
        labels = np.concatenate((labels,cnt*np.ones((len(labels),1))),axis=1)
        label_vec = np.concatenate((label_vec,labels),axis=0)
        cnt = cnt + 1  
print('Annotations loaded!')

# plot
fig = plt.figure(figsize=(num_classes[1], num_classes[0]), dpi=80)
np.random.seed(42)
img_size = 224
for organ in range(0,num_classes[0]):
    for modality in range(0,num_classes[1]):
        curr_label = [organ,modality]
        curr_list = np.where(np.sum(label_vec == curr_label,axis=1) == 2)[0]
        if len(curr_list) > 0:
            # get a random number
            idx = np.random.randint(1,len(curr_list))
            curr_img = imgs[curr_list[idx],:,:]
            if curr_img.max() != 0:
                curr_img = curr_img*(255.0/curr_img.max())
            curr_img = np.stack((curr_img,)*3, axis=-1)
            img = Image.fromarray(curr_img.astype(np.uint8))
        else:
            continue
        img = transforms.Resize(size=img_size)(img)
            
        # plot
        cnt = organ*num_classes[1] + modality + 1
        plt.subplot(num_classes[0],num_classes[1],cnt)
        plt.imshow(img)
        plt.axis('off') 
plt.subplots_adjust(wspace=0, hspace=0)     
plt.show()            

## Configs for hyper experiments

In [ ]:
from config_experiments_medmnist import config_experiments_hyper
import json

# hyperparameter configs
config_experiments_hyper(results_path + '/MedMNIST/results/results_Organ_hyper/',True)
with open(results_path + '/MedMNIST/results/results_Organ_hyper/' + 'configs/' + str(10) + '.json') as config_file:
        config = json.load(config_file)
        print(config)          

## run from terminal - hyper experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Organ_hyper' --gpu_id=0

## Configs for seed experiments

In [ ]:
from config_experiments_medmnist import config_experiments
import json

# hyperparameter configs
config_experiments(results_path + '/MedMNIST/results/results_Organ/',True)
with open(results_path + 'MedMNIST/results/results_Organ/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)           

## run from terminal - seed experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Organ' --gpu_id=0

## Configs for distribution experiments

In [ ]:
from config_experiments_medmnist import config_experiments_dist
import json

# hyperparameter configs
config_experiments_dist(results_path + '/MedMNIST/results/results_Organ_dist/',True)
with open(results_path + '/MedMNIST/results/results_Organ_dist/' + 'configs/' + str(20) + '.json') as config_file:
        config = json.load(config_file)
        print(config)         

## run from terminal - dist experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Organ_dist' --gpu_id=0

## Configs for secondary experiments

In [ ]:
from config_experiments_medmnist import config_experiments_secondary
import json

# hyperparameter configs
config_experiments_secondary(results_path + '/MedMNIST/results/results_Secondary/',True)
with open(results_path + '/MedMNIST/results/results_Secondary/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)           

## run from terminal - secondary experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Secondary' --gpu_id=0

## Configs for distribution secondary experiments

In [ ]:
from config_experiments_medmnist import config_experiments_secondary_dist
import json

# hyperparameter configs
config_experiments_secondary_dist(results_path + '/MedMNIST/results/results_Secondary_dist/',True)
with open(results_path + '/MedMNIST/results/results_Secondary_dist/' + 'configs/' + str(60) + '.json') as config_file:
        config = json.load(config_file)
        print(config)         

## run from terminal - dist secondary experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Secondary_dist' --gpu_id=0

## Configs for reduced experiments

In [ ]:
from config_experiments_medmnist import config_experiments_reduced
import json

# hyperparameter configs
config_experiments_reduced(results_path + '/MedMNIST/results/results_Reduced/',True)
with open(results_path + '/MedMNIST/results/results_Reduced/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)           

## run from terminal - reduced experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='Reduced' --gpu_id=0

## Configs for reduced secondary experiments

In [ ]:
from config_experiments_medmnist import config_experiments_reduced_secondary
import json

# hyperparameter configs
config_experiments_reduced_secondary(results_path + '/MedMNIST/results/results_ReducedSecondary/',True)
with open(results_path + '/MedMNIST/results/results_ReducedSecondary/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)    

## run from terminal - reduced secondary experiments

In [ ]:
!python main_medmnist.py --experiment_id=0 --task_to_run='ReducedSecondary' --gpu_id=0

## check how many experiments are done - hyper experiments

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_hyper
from utils import gatherMetrics_medmnist
import json

# current task
task = 'Organ_hyper'
    
experiment_list = config_experiments_hyper(results_path + '/MedMNIST/results/results_' + task + '/',False)

cnt = 0
# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        # print(curr_exp)
        cnt = cnt + 1
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments and print best experiment
df_hyper = gatherMetrics_medmnist(experiment_list, task)  
idx = np.argmax(df_hyper['val_bal_acc'])
print(df_hyper['learning_rate'][idx])

## check how many experiments are done - seed experiments

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments
from utils import gatherMetrics_medmnist
import json

# current task
task = 'Organ'
    
experiment_list = config_experiments(results_path + '/MedMNIST/results/results_' + task + '/',False)

cnt = 0
# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        # print(curr_exp)
        cnt = cnt + 1
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments and print best experiment
df = gatherMetrics_medmnist(experiment_list, task)  

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_dist
from utils import gatherMetrics_medmnist
import json

# current task
task = 'Organ_dist'
    
experiment_list = config_experiments_dist(results_path + '/MedMNIST/results/results_' + task + '/',False)

cnt = 0
# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        # print(curr_exp)
        cnt = cnt + 1
    # else:
    #     print(curr_exp)
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments and print best experiment
df_dist = gatherMetrics_medmnist(experiment_list, task)  

## check how many experiments are done - secondary experiments

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_secondary
from utils import gatherMetrics_medmnist
import json

task = 'Secondary'

experiment_list = config_experiments_secondary(results_path + '/MedMNIST/results/results_'+ task +'/',False)

cnt = 0

# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        cnt = cnt + 1
    else:
        print(curr_path)
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments
df_secondary = gatherMetrics_medmnist(experiment_list, task)

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_secondary_dist
from utils import gatherMetrics_medmnist
import json

task = 'Secondary_dist'

experiment_list = config_experiments_secondary_dist(results_path + '/MedMNIST/results/results_'+ task +'/',False)

cnt = 0

# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        cnt = cnt + 1
    # else:
    #     print(curr_path)
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments
df_dist_secondary = gatherMetrics_medmnist(experiment_list, task)

## check how many experiments are done - reduced experiments

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_reduced
from utils import gatherMetrics_medmnist
import json

task = 'Reduced'

experiment_list = config_experiments_reduced(results_path + '/MedMNIST/results/results_'+ task +'/',False)

cnt = 0

# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        cnt = cnt + 1
    # else:
    #     print(curr_path)
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments
df_reduced = gatherMetrics_medmnist(experiment_list, task)

## check how many experiments are done - reduced secondary experiments

In [ ]:
# check how many experiments are done
import os
import numpy as np
from config_experiments_medmnist import config_experiments_reduced_secondary
from utils import gatherMetrics_medmnist
import json

task = 'ReducedSecondary'

experiment_list = config_experiments_reduced_secondary(results_path + '/MedMNIST/results/results_'+ task +'/',False)

cnt = 0

# iterate over experiments
for i in range(len(experiment_list)):
    curr_exp = experiment_list[i]['experiment_id']
    
    # check if exists
    curr_path = results_path + '/MedMNIST/results/results_' + task + '/experiment_' + str(curr_exp) + '/checkpoint_last' + str(experiment_list[i]['num_epochs']-1)
    if os.path.exists(curr_path):
        cnt = cnt + 1
    else:
        print(curr_path)
print('Current status of done experiments: ' + str(cnt) + '/' + str(len(experiment_list)))        

# gather all experiments
df_reduced_secondary = gatherMetrics_medmnist(experiment_list, task)

## plot accuracy difference - secondary experiments

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# plot for config['sample_class']
data_dir = data_path + '/Data/MedMNIST/'
results_dir = results_path + '/MedMNIST/results/results_Organ/'
results_dir_sec = results_path + '/MedMNIST/results/results_Secondary/'

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))
        
# x and y_axis ticks
y_list = CLASS_NAMES_exp[0]  
x_list = CLASS_NAMES_exp[1]  

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

# plot params
plotOOD = False

# iterate over seeds
seed_vec = [42,73,666,777,1009]# ,1279,1597,1811,1949,2053]
model_type_vec = ['ResNet18'] # ,'DenseNet121','ResNet50','ViTb16'
sampling_percentage_vec = [100,75,50,35,25,10,5]

# colors for plotting
blue_colors = [mcolors.CSS4_COLORS['darkslateblue'],
               mcolors.CSS4_COLORS['midnightblue'],
                mcolors.CSS4_COLORS['darkblue'],
                mcolors.CSS4_COLORS['mediumblue'],
                mcolors.CSS4_COLORS['royalblue'],
                mcolors.CSS4_COLORS['cornflowerblue'],
                mcolors.CSS4_COLORS['lightblue']]

red_colors = [mcolors.CSS4_COLORS['darkred'],
                  mcolors.CSS4_COLORS['firebrick'],
                  mcolors.CSS4_COLORS['crimson'],
                  mcolors.CSS4_COLORS['red'],
                  mcolors.CSS4_COLORS['orangered'],
                  mcolors.CSS4_COLORS['orange'],
                  mcolors.CSS4_COLORS['gold']]

purple_colors = [mcolors.CSS4_COLORS['indigo'],
                mcolors.CSS4_COLORS['purple'],
                mcolors.CSS4_COLORS['darkviolet'],
                mcolors.CSS4_COLORS['mediumorchid'],
                mcolors.CSS4_COLORS['orchid'],
                mcolors.CSS4_COLORS['thistle'],
                mcolors.CSS4_COLORS['lavender']]

# iterate over random seed
for model_type in model_type_vec:
    print('Model type: ' + model_type)
    
    # initialize
    mean_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_mat_ = np.zeros([len(sampling_percentage_vec),num_classes[1]])
    mean_eval_sec_mat_ = np.zeros([len(sampling_percentage_vec),num_classes[1]])
    std_eval_mat_ = np.zeros([len(sampling_percentage_vec),num_classes[1]])
    std_eval_sec_mat_ = np.zeros([len(sampling_percentage_vec),num_classes[1]])
    cnt_sampling_percentage = 0
    
    # iterate over sampling percentages
    for sampling_percentage in sampling_percentage_vec:
        cnt_seed = 0
    
        # initialize
        eval_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_sec_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_vec_ = np.zeros([len(seed_vec),num_classes[1]])
        eval_sec_vec_ = np.zeros([len(seed_vec),num_classes[1]])
        eval_vec = np.zeros([len(seed_vec),1])
        eval_sec_vec = np.zeros([len(seed_vec),1])
        cnt_mat = np.zeros([num_classes[0],num_classes[1]])
    
        for seed in seed_vec:
            
            # get the current experiment
            df_ = df[(df['seed'] == seed) & (df['model_type'] == model_type) & (df['sampling_percentage'] == sampling_percentage) ]
            
            # get the current experiment
            curr_exp = int(df_['experiment_id'])
            curr_path = results_dir + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
            if os.path.exists(curr_path):
                with open(curr_path, 'r') as f:
                    data = json.load(f) 
                eval_ = np.array(data['eval_mat_sec']) 
    
            # filter df_nbrData_secondary
            eval_sec = np.zeros([len(CLASS_NAMES[0]),len(CLASS_NAMES[1])])
    
            # iterate over secondary experiments
            cnt_sec = 0
            for i in range(num_classes[1]):
        
                # get the current experiment
                df_secondary_ = df_secondary[(df_secondary['model_type'] == model_type) &
                                             (df_secondary['sampling_percentage'] == sampling_percentage) &
                                             (df_secondary['task2'] == i) & 
                                             (df_secondary['seed'] == seed)]
                curr_exp = int(df_secondary_['experiment_id'])
                curr_path = results_dir_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                if os.path.exists(curr_path):
                    with open(curr_path, 'r') as f:
                        data = json.load(f) 
                    # calculate id accuracy 
                    for j in range(num_classes[0]):
                        eval_sec[j,i] = np.array(data['eval_mat'])[j]  
                    
            # sum for each seed
            eval_mat = eval_mat + eval_
            eval_sec_mat = eval_sec_mat + eval_sec
            eval_vec_[cnt_seed,:] = np.nanmean(eval_,axis=0)
            eval_sec_vec_[cnt_seed,:] = np.nanmean(eval_sec,axis=0)
            eval_vec[cnt_seed,:] = np.nanmean(eval_)
            eval_sec_vec[cnt_seed,:] = np.nanmean(eval_sec)
            cnt_mat = cnt_mat + ((eval_sec - eval_) > 0)
            cnt_seed = cnt_seed + 1
               
        # average
        eval_mat = eval_mat/cnt_seed
        eval_sec_mat = eval_sec_mat/cnt_seed
        # detailed
        mean_eval_mat_[cnt_sampling_percentage,:] = np.mean(eval_vec_,axis=0)
        mean_eval_sec_mat_[cnt_sampling_percentage,:] = np.mean(eval_sec_vec_,axis=0)
        std_eval_mat_[cnt_sampling_percentage,:] = np.nanstd(eval_vec_,axis=0)
        std_eval_sec_mat_[cnt_sampling_percentage,:] = np.nanstd(eval_sec_vec_,axis=0)
        # general
        mean_eval_mat[cnt_sampling_percentage,:] = np.mean(eval_vec)
        mean_eval_sec_mat[cnt_sampling_percentage,:] = np.mean(eval_sec_vec)
        std_eval_mat[cnt_sampling_percentage,:] = np.nanstd(eval_vec)
        std_eval_sec_mat[cnt_sampling_percentage,:] = np.nanstd(eval_sec_vec)
        
        # print
        if plotOOD:
            print('Sampling percentage: ' + str(sampling_percentage))
            print('General model - mean & std: ' + str(mean_eval_mat[cnt_sampling_percentage,:]) + '&' + str(std_eval_mat[cnt_sampling_percentage,:]))
            print('Specialized model - mean & std: ' + str(mean_eval_sec_mat[cnt_sampling_percentage,:]) + '&' + str(std_eval_sec_mat[cnt_sampling_percentage,:]))
        cnt_sampling_percentage = cnt_sampling_percentage + 1
        
        # number of datapoints
        datapoints_train = np.array([[1956,1153,1148],
         [1408, 626, 637],
         [1359, 608, 615],
         [1474, 600, 721],
         [3963,1088,1132],
         [3817,1170,1119],
         [6164,2986,3464],
         [3919,1002, 741],
         [3929,1022, 803],
         [3031,1173,2004],
         [3561,1572,1556]])
        datapoints_train = datapoints_train * (sampling_percentage/100)

        # datapoints_train = np.array([[1000,1000,1000],
        #  [1000, 626, 637],
        #  [1000, 608, 615],
        #  [1000, 600, 721],
        #  [1000,1000,1000],
        #  [1000,1000,1000],
        #  [1000,1000,1000],
        #  [1000,1000, 741],
        #  [1000,1000, 803],
        #  [1000,1000,1000],
        #  [1000,1000,1000]])

        if plotOOD:    
    
            # diff ood
            eval_diff = eval_mat - eval_sec_mat

            # plot
            fig = plt.figure(figsize=(12, 4), dpi=80)
    
            # number of datapoints
            ax1 = fig.add_subplot(1,5,1)
            im1 = ax1.imshow(datapoints_train) 
            ax1.set_xticks(np.arange(len(x_list)), labels=x_list)
            ax1.set_yticks(np.arange(len(y_list)), labels=y_list)
            plt.setp(ax1.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
            plt.setp(ax1.get_yticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
            ax1.set_title("Number of samples")
            fig.colorbar(im1, orientation='vertical')

            # specialized model
            ax2 = fig.add_subplot(1,5,2)
            im2 = ax2.imshow(eval_sec_mat, vmin=0, vmax=1) 
            ax2.set_xticks(np.arange(len(x_list)), labels=x_list)
            ax2.set_yticks([])
            plt.setp(ax2.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
            plt.setp(ax2.get_yticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
            ax2.set_title("Specialized model")
            fig.colorbar(im2, orientation='vertical')

            # test
            ax3 = fig.add_subplot(1,5,3)
            im3 = ax3.imshow(eval_mat, vmin=0, vmax=1) 
            ax3.set_xticks(np.arange(len(x_list)),labels=x_list)
            ax3.set_yticks([])
            plt.setp(ax3.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
            plt.setp(ax3.get_yticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
            ax3.set_title("General model")
            fig.colorbar(im3, orientation='vertical')

            # difference
            ax4 = fig.add_subplot(1,5,4)
            im4 = ax4.imshow(eval_diff, vmin=-0.1, vmax=0.1, cmap=cmap) 
            ax4.set_xticks(np.arange(len(x_list)),labels=x_list)
            ax4.set_yticks([])
            plt.setp(ax4.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
            plt.setp(ax4.get_yticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
            ax4.set_title("Difference")
            fig.colorbar(im4, orientation='vertical')
    
            # check how many times specialized better than general
            ax5 = fig.add_subplot(1,5,5)
            im5 = ax5.imshow(cnt_mat, vmin=0, vmax=len(seed_vec)) 
            ax5.set_xticks(np.arange(len(x_list)),labels=x_list)
            ax5.set_yticks([])
            plt.setp(ax5.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
            plt.setp(ax5.get_yticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
            ax5.set_title("Specialized better")
            fig.colorbar(im5, orientation='vertical')

            plt.subplots_adjust(wspace=0.1)
            plt.show()
            
    fig = plt.figure(figsize=(4.3, 3), dpi=80)
    # plt.subplot(1,2,1)
    plt.errorbar(sampling_percentage_vec,mean_eval_sec_mat[:,0],yerr=std_eval_sec_mat[:,0],linewidth=2.5,label='Specialized',color=blue_colors[0])
    plt.errorbar(sampling_percentage_vec,mean_eval_mat[:,0],yerr=std_eval_mat[:,0],linewidth=2.5,label='Multi-domain',color=red_colors[0])
    plt.ylim([0.5,1])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Average balanced accuracy')
    # plt.title('Specialized vs General -  OOD')
    plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
    # plt.show()
    # plt.savefig('figs/medmnist_specialized_vs_general.eps', bbox_inches='tight', format='eps')  
    plt.savefig('figs/medmnist_specialized_vs_general.pdf', bbox_inches='tight', format='pdf')  

    if plotOOD:        
        # plot
        fig = plt.figure(figsize=(13, 3), dpi=80)
        plt.subplot(1,2,1)
        plt.plot(sampling_percentage_vec,mean_eval_sec_mat,label='Specialized',color=blue_colors[0])
        for i in range(0,num_classes[1]):
            if i == 0:
                view = 'Axial'
            elif i == 1:
                view = 'Coronal'
            else:
                view = 'Sagittal'
            plt.plot(sampling_percentage_vec,mean_eval_sec_mat_[:,i],label='Specialized - View: ' + view,linestyle='dashed',color=blue_colors[2*i+1])
        plt.plot(sampling_percentage_vec,mean_eval_mat,label='General',color=red_colors[0])
        for i in range(0,num_classes[1]):
            if i == 0:
                view = 'Axial'
            elif i == 1:
                view = 'Coronal'
            else:
                view = 'Sagittal'
            plt.plot(sampling_percentage_vec,mean_eval_mat_[:,i],label='General - View: ' + view,linestyle='dashed',color=red_colors[2*i+1])
        plt.ylim([0.5,1])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Average balanced accuracy')
        plt.title('Specialized vs General')
        plt.legend(bbox_to_anchor=(1.75, 0.5),loc='right', ncol=1)
        plt.subplot(1,2,2)
        plt.plot(sampling_percentage_vec,mean_eval_mat-mean_eval_sec_mat,color=purple_colors[0],label='Average')
        for i in range(0,num_classes[1]):
            if i == 0:
                view = 'Axial'
            elif i == 1:
                view = 'Coronal'
            else:
                view = 'Sagittal'
            plt.plot(sampling_percentage_vec,mean_eval_mat_[:,i]-mean_eval_sec_mat_[:,i],label='View: ' + view,linestyle='dashed',color=purple_colors[2*i+1])
        plt.ylim([-0.05,0.05])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Balanced accuracy difference')
        plt.title('Difference - Specialized vs General')
        plt.legend(bbox_to_anchor=(1.475, 0.5),loc='right', ncol=1)
        plt.subplots_adjust(wspace=1)
        plt.show()  

## plot accuracy difference - dist experiments

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import matplotlib.colors as mcolors

# plot for config['sample_class']
data_dir = data_path + '/Data/MedMNIST/'
results_dir = results_path + '/MedMNIST/results/results_Organ_dist/'
results_dir_sec = results_path + '/MedMNIST/results/results_Secondary_dist/'

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))
        
# x and y_axis ticks
y_list = CLASS_NAMES_exp[0]  
x_list = CLASS_NAMES_exp[1]  

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

# iterate over seeds
seed_vec = [42,73,666,777,1009]# ,1279,1597,1811,1949,2053]
model_type_vec = ['ResNet18'] # ,'DenseNet121' 
sampling_percentage_vec = [100,75,50,35,25,10,5]
max_sample_train = 600

blue_colors = [mcolors.CSS4_COLORS['darkslateblue'],
               mcolors.CSS4_COLORS['midnightblue'],
                mcolors.CSS4_COLORS['darkblue'],
                mcolors.CSS4_COLORS['mediumblue'],
                mcolors.CSS4_COLORS['royalblue'],
                mcolors.CSS4_COLORS['cornflowerblue'],
                mcolors.CSS4_COLORS['lightblue']]

red_colors = [mcolors.CSS4_COLORS['darkred'],
                  mcolors.CSS4_COLORS['firebrick'],
                  mcolors.CSS4_COLORS['crimson'],
                  mcolors.CSS4_COLORS['red'],
                  mcolors.CSS4_COLORS['orangered'],
                  mcolors.CSS4_COLORS['orange'],
                  mcolors.CSS4_COLORS['gold']]

purple_colors = [mcolors.CSS4_COLORS['indigo'],
                mcolors.CSS4_COLORS['purple'],
                mcolors.CSS4_COLORS['darkviolet'],
                mcolors.CSS4_COLORS['mediumorchid'],
                mcolors.CSS4_COLORS['orchid'],
                mcolors.CSS4_COLORS['thistle'],
                mcolors.CSS4_COLORS['lavender']]

# iterate over random seed
for model_type in model_type_vec:
    print('Model type: ' + model_type)

    # initialize
    mean_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
    cnt_sampling_percentage = 0
    
    # iterate over sampling percentage
    for sampling_percentage in sampling_percentage_vec:
        # print
        print('Sampling percentage: ' + str(sampling_percentage))
    
        # initlaize eval_mat
        eval_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_id_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_vec = np.zeros([len(seed_vec),1])
        eval_id_vec = np.zeros([len(seed_vec),1])
        eval_sec_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_sec_id_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_sec_vec = np.zeros([len(seed_vec),1])
        eval_sec_id_vec = np.zeros([len(seed_vec),1])
        cnt_seed = 0
    
        # iterate over random seed
        for seed in seed_vec:     
        
            # get the current general experiment
            df_ = df_dist[(df_dist['seed'] == seed) & 
                     (df_dist['model_type'] == model_type) &
                     (df_dist['max_sample_train'] == max_sample_train) &
                     (df_dist['sampling_percentage'] == sampling_percentage)]
            curr_exp = int(df_['experiment_id'])
            curr_path = results_dir + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
            if os.path.exists(curr_path):
                with open(curr_path, 'r') as f:
                    data = json.load(f) 
                eval_ = np.array(data['eval_mat_sec'])
                eval_id = np.zeros([num_classes[0],num_classes[1]])
            
                # calculate id accuracy 
                for j in range(num_classes[0]):
                    for i in range(num_classes[1]):
                        tmp = np.array(data['eval_mat_sec'], copy=True)  
                        tmp[j,i] = float('nan')
                        eval_id[j,i] = np.nanmean(tmp) 
                    
            # sum for each seed
            eval_mat = eval_mat + eval_
            eval_id_mat = eval_id_mat + eval_id
            eval_vec[cnt_seed] = np.nanmean(eval_)
            eval_id_vec[cnt_seed] = np.nanmean(eval_id)
    
            # filter df_nbrData_secondary
            eval_sec = np.zeros([num_classes[0],num_classes[1]])
            eval_sec_id = np.zeros([num_classes[0],num_classes[1]])
    
            # iterate over secondary experiments
            for i in range(num_classes[1]):
        
                # get the current experiment
                df_sec_ = df_dist_secondary[(df_dist_secondary['task2'] == i) & 
                                   (df_dist_secondary['seed'] == seed) & 
                                   (df_dist_secondary['model_type'] == model_type) & 
                                   (df_dist_secondary['max_sample_train'] == max_sample_train) &
                                   (df_dist_secondary['sampling_percentage'] == sampling_percentage)]
                curr_exp = int(df_sec_['experiment_id'])
                curr_path = results_dir_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                if os.path.exists(curr_path):
                    with open(curr_path, 'r') as f:
                        data = json.load(f) 
                
                    # calculate id accuracy 
                    for j in range(num_classes[0]):
                        eval_sec[j,i] = np.array(data['eval_mat'])[j]
                        tmp = np.array(data['eval_mat'], copy=True)  
                        tmp[j] = float('nan')
                        eval_sec_id[j,i] = np.nanmean(tmp) 
                    
            # sum for each seed
            eval_sec_mat = eval_sec_mat + eval_sec
            eval_sec_id_mat = eval_sec_id_mat + eval_sec_id
            eval_sec_vec[cnt_seed] = np.nanmean(eval_sec)
            eval_sec_id_vec[cnt_seed] = np.nanmean(eval_sec_id)
            cnt_seed = cnt_seed + 1
        
        # assign for each sampling_percentage
        # general
        eval_mat = eval_mat/cnt_seed
        eval_id_mat = eval_id_mat/cnt_seed
        mean_eval_mat[cnt_sampling_percentage] = np.nanmean(eval_vec)
        mean_eval_id_mat[cnt_sampling_percentage] = np.nanmean(eval_id_vec)
        std_eval_mat[cnt_sampling_percentage] = np.nanstd(eval_vec)
        std_eval_id_mat[cnt_sampling_percentage] = np.nanstd(eval_id_vec)
        # secondary
        eval_sec_mat = eval_sec_mat/cnt_seed
        eval_sec_id_mat = eval_sec_id_mat/cnt_seed
        mean_eval_sec_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_vec)
        mean_eval_sec_id_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_id_vec)
        std_eval_sec_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_vec)
        std_eval_sec_id_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_id_vec)
        cnt_sampling_percentage = cnt_sampling_percentage + 1
    
    fig = plt.figure(figsize=(4.3, 3), dpi=80)
    # plt.subplot(1,2,1)
    plt.errorbar(sampling_percentage_vec,mean_eval_sec_mat[:,0],yerr=std_eval_sec_mat[:,0],linewidth=2.5,label='Specialized',color=blue_colors[0])
    plt.errorbar(sampling_percentage_vec,mean_eval_mat[:,0],yerr=std_eval_mat[:,0],linewidth=2.5,label='Multi-domain',color=red_colors[0])
    plt.ylim([0.5,1])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Average balanced accuracy')
    # plt.title('Specialized vs General -  OOD')
    plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
    # plt.show()
    # plt.savefig('figs/medmnist_dist_specialized_vs_general.eps', bbox_inches='tight', format='eps')
    plt.savefig('figs/medmnist_dist_specialized_vs_general.pdf', bbox_inches='tight', format='pdf')

## plot accuracy difference - all reduced experiments

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import matplotlib.colors as mcolors

# plot for config['sample_class']
data_dir = data_path + '/Data/MedMNIST/'
results_dir = results_path + '/MedMNIST/results/results_Organ/'
results_dir_red = results_path + '/MedMNIST/results/results_Reduced/'
results_dir_sec = results_path + '/MedMNIST/results/results_Secondary/'
results_dir_red_sec = results_path + '/MedMNIST/results/results_ReducedSecondary/'

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))
        
# x and y_axis ticks
y_list = CLASS_NAMES_exp[0]  
x_list = CLASS_NAMES_exp[1]  

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

# iterate over seeds
seed_vec = [42,73,666,777,1009]# ,1279,1597,1811,1949,2053]
model_type_vec = ['ResNet18'] # ,'DenseNet121' 
sampling_percentage_vec = [100,75,50,35,25,10,5]
reduced_percentage_vec = [25,50,75,85,95,100] 

blue_colors = [mcolors.CSS4_COLORS['darkslateblue'],
               mcolors.CSS4_COLORS['midnightblue'],
                mcolors.CSS4_COLORS['darkblue'],
                mcolors.CSS4_COLORS['mediumblue'],
                mcolors.CSS4_COLORS['royalblue'],
                mcolors.CSS4_COLORS['cornflowerblue'],
                mcolors.CSS4_COLORS['lightblue']]

red_colors = [mcolors.CSS4_COLORS['darkred'],
                  mcolors.CSS4_COLORS['firebrick'],
                  mcolors.CSS4_COLORS['crimson'],
                  mcolors.CSS4_COLORS['red'],
                  mcolors.CSS4_COLORS['orangered'],
                  mcolors.CSS4_COLORS['orange'],
                  mcolors.CSS4_COLORS['gold']]

purple_colors = [mcolors.CSS4_COLORS['indigo'],
                mcolors.CSS4_COLORS['purple'],
                mcolors.CSS4_COLORS['darkviolet'],
                mcolors.CSS4_COLORS['mediumorchid'],
                mcolors.CSS4_COLORS['orchid'],
                mcolors.CSS4_COLORS['thistle'],
                mcolors.CSS4_COLORS['lavender']]

# iterate over random seed
for model_type in model_type_vec:
    print('Model type: ' + model_type)

    # initialize
    mean_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_red_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    mean_eval_red_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    std_eval_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_red_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    std_eval_red_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    mean_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
    mean_eval_red_sec_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    mean_eval_red_sec_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    std_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
    std_eval_red_sec_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    std_eval_red_sec_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
    cnt_sampling_percentage = 0
    
    # iterate over sampling percentage
    for sampling_percentage in sampling_percentage_vec:
        # print
        print('Sampling percentage: ' + str(sampling_percentage))
    
        # initlaize eval_mat
        eval_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_id_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_vec = np.zeros([len(seed_vec),1])
        eval_id_vec = np.zeros([len(seed_vec),1])
        eval_sec_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_sec_id_mat = np.zeros([num_classes[0],num_classes[1]])
        eval_sec_vec = np.zeros([len(seed_vec),1])
        eval_sec_id_vec = np.zeros([len(seed_vec),1])
        cnt_seed = 0
    
        # iterate over random seed
        for seed in seed_vec:     
        
            # get the current general experiment
            df_ = df[(df['seed'] == seed) & 
                     (df['model_type'] == model_type) &
                     (df['sampling_percentage'] == sampling_percentage)]
            curr_exp = int(df_['experiment_id'])
            curr_path = results_dir + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
            if os.path.exists(curr_path):
                with open(curr_path, 'r') as f:
                    data = json.load(f) 
                
                eval_ = np.array(data['eval_mat_sec'])
                eval_id = np.zeros([num_classes[0],num_classes[1]])
            
                # calculate id accuracy 
                for j in range(num_classes[0]):
                    for i in range(num_classes[1]):
                        tmp = np.array(data['eval_mat_sec'], copy=True)  
                        tmp[j,i] = float('nan')
                        eval_id[j,i] = np.nanmean(tmp) 
                    
            # sum for each seed
            eval_mat = eval_mat + eval_
            eval_id_mat = eval_id_mat + eval_id
            eval_vec[cnt_seed] = np.nanmean(eval_)
            eval_id_vec[cnt_seed] = np.nanmean(eval_id)
    
            # filter df_nbrData_secondary
            eval_sec = np.zeros([num_classes[0],num_classes[1]])
            eval_sec_id = np.zeros([num_classes[0],num_classes[1]])
    
            # iterate over secondary experiments
            for i in range(num_classes[1]):
        
                # get the current experiment
                df_sec_ = df_secondary[(df_secondary['task2'] == i) & 
                                   (df_secondary['seed'] == seed) & 
                                   (df_secondary['model_type'] == model_type) & 
                                   (df_secondary['sampling_percentage'] == sampling_percentage)]
                curr_exp = int(df_sec_['experiment_id'])
                curr_path = results_dir_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                if os.path.exists(curr_path):
                    with open(curr_path, 'r') as f:
                        data = json.load(f) 
                
                    # calculate id accuracy 
                    for j in range(num_classes[0]):
                        eval_sec[j,i] = np.array(data['eval_mat'])[j]
                        tmp = np.array(data['eval_mat'], copy=True)  
                        tmp[j] = float('nan')
                        eval_sec_id[j,i] = np.nanmean(tmp) 
                    
            # sum for each seed
            eval_sec_mat = eval_sec_mat + eval_sec
            eval_sec_id_mat = eval_sec_id_mat + eval_sec_id
            eval_sec_vec[cnt_seed] = np.nanmean(eval_sec)
            eval_sec_id_vec[cnt_seed] = np.nanmean(eval_sec_id)
            cnt_seed = cnt_seed + 1
        
        # assign for each sampling_percentage
        # general
        eval_mat = eval_mat/cnt_seed
        eval_id_mat = eval_id_mat/cnt_seed
        mean_eval_mat[cnt_sampling_percentage] = np.nanmean(eval_vec)
        mean_eval_id_mat[cnt_sampling_percentage] = np.nanmean(eval_id_vec)
        std_eval_mat[cnt_sampling_percentage] = np.nanstd(eval_vec)
        std_eval_id_mat[cnt_sampling_percentage] = np.nanstd(eval_id_vec)
        # secondary
        eval_sec_mat = eval_sec_mat/cnt_seed
        eval_sec_id_mat = eval_sec_id_mat/cnt_seed
        mean_eval_sec_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_vec)
        mean_eval_sec_id_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_id_vec)
        std_eval_sec_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_vec)
        std_eval_sec_id_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_id_vec)
    
        # iterate over reduced secondary experiments
        cnt_reduced_percentage = 0
    
        for reduced_percentage in reduced_percentage_vec:
            # initialize
            eval_red_mat = np.zeros([num_classes[0],num_classes[1]])
            eval_red_id_mat = np.zeros([num_classes[0],num_classes[1]])
            eval_red_vec = np.zeros([len(seed_vec),1])
            eval_red_id_vec = np.zeros([len(seed_vec),1])
            eval_red_sec_mat = np.zeros([num_classes[0],num_classes[1]])
            eval_red_sec_id_mat = np.zeros([num_classes[0],num_classes[1]])
            eval_red_sec_vec = np.zeros([len(seed_vec),1])
            eval_red_sec_id_vec = np.zeros([len(seed_vec),1])
            cnt_seed = 0
        
            # iterate over random seed
            for seed in seed_vec: 
            
                # initialize
                eval_red = np.zeros([num_classes[0],num_classes[1]])
                eval_red_id = np.zeros([num_classes[0],num_classes[1]])
            
                # get the current experiment
                df_red = df_reduced[(df_reduced['seed'] == seed) & 
                                (df_reduced['model_type'] == model_type) & 
                                (df_reduced['sampling_percentage'] == sampling_percentage) & 
                                (df_reduced['reduced_percentage'] == reduced_percentage)]
            
                # iterate over tasks
                for i in range(num_classes[1]):
                    for j in range(num_classes[0]):
                        # get the current experiment
                        df_red_ = df_red[(df_red['task1'] == j) & (df_red['task2'] == i)]
                        curr_exp = int(df_red_['experiment_id'])
                        curr_path = results_dir_red + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                        if os.path.exists(curr_path):
                            with open(curr_path, 'r') as f:
                                data = json.load(f) 
                
                            eval_red[j,i] = np.array(data['eval_mat_sec'])[j,i]
                            tmp = np.array(data['eval_mat_sec'], copy=True)  
                            tmp[j,i] = float('nan')
                            eval_red_id[j,i] = np.nanmean(tmp)   
                # sum for each seed
                eval_red_mat = eval_red_mat + eval_red
                eval_red_id_mat = eval_red_id_mat + eval_red_id
                eval_red_vec[cnt_seed] = np.nanmean(eval_red)
                eval_red_id_vec[cnt_seed] = np.nanmean(eval_red_id)
            
                # initialize
                eval_red_sec = np.zeros([num_classes[0],num_classes[1]])
                eval_red_sec_id = np.zeros([num_classes[0],num_classes[1]])
            
                # get the current experiment
                df_red_sec = df_reduced_secondary[(df_reduced_secondary['seed'] == seed) & 
                                             (df_reduced_secondary['model_type'] == model_type) &      
                                             (df_reduced_secondary['sampling_percentage'] == sampling_percentage) & 
                                             (df_reduced_secondary['reduced_percentage'] == reduced_percentage)]
            
                # iterate over tasks
                for i in range(num_classes[1]):
                    # initialize cnt
                    for j in range(num_classes[0]):
                        # get the current experiment
                        df_red_sec_ = df_red_sec[(df_red_sec['task1'] == j) & (df_red_sec['task2'] == i)]
                        curr_exp = int(df_red_sec_['experiment_id'])
                        curr_path = results_dir_red_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                        if os.path.exists(curr_path):
                            with open(curr_path, 'r') as f:
                                data = json.load(f) 
                
                            eval_red_sec[j,i] = np.array(data['eval_mat'])[j]
                            tmp = np.array(data['eval_mat'], copy=True)  
                            tmp[j] = float('nan')
                            eval_red_sec_id[j,i] = np.nanmean(tmp)  
                    
                # sum for each seed
                eval_red_sec_mat = eval_red_sec_mat + eval_red_sec
                eval_red_sec_id_mat = eval_red_sec_id_mat + eval_red_sec_id
                eval_red_sec_vec[cnt_seed] = np.nanmean(eval_red_sec)
                eval_red_sec_id_vec[cnt_seed] = np.nanmean(eval_red_sec_id)
                cnt_seed = cnt_seed + 1   
            
            # assign for each sampling_percentage
            # general
            eval_red_mat = eval_red_mat/cnt_seed
            eval_red_id_mat = eval_red_id_mat/cnt_seed
            mean_eval_red_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_vec)
            mean_eval_red_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_id_vec)
            std_eval_red_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_vec)
            std_eval_red_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_id_vec)
            # secondary
            eval_red_sec_mat = eval_red_sec_mat/cnt_seed
            eval_red_sec_id_mat = eval_red_sec_id_mat/cnt_seed
            mean_eval_red_sec_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_sec_vec)
            mean_eval_red_sec_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_sec_id_vec)
            std_eval_red_sec_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_sec_vec)
            std_eval_red_sec_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_sec_id_vec)
            cnt_reduced_percentage = cnt_reduced_percentage + 1   
    
        # increment cnt for sampling percentage
        cnt_sampling_percentage = cnt_sampling_percentage + 1   
    
    fig = plt.figure(figsize=(5.25, 3), dpi=80)
    # plt.subplot(1,2,1)
    plt.errorbar(sampling_percentage_vec,mean_eval_sec_mat[:,0],yerr=std_eval_sec_mat[:,0],label='Specialized - '+ '0%',color=blue_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.errorbar(sampling_percentage_vec,mean_eval_red_sec_mat[:,i],yerr=std_eval_red_sec_mat[:,i],label='Specialized - ' + str(reduced_percentage_vec[i])+'%',color=blue_colors[i+1])
    plt.errorbar(sampling_percentage_vec,mean_eval_mat[:,0],yerr=std_eval_mat[:,0],label='Multi-domain - '+ '0%',color=red_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.errorbar(sampling_percentage_vec,mean_eval_red_mat[:,i],yerr=std_eval_red_mat[:,i],label='Multi-domain - ' + str(reduced_percentage_vec[i])+'%',color=red_colors[i+1])
    plt.ylim([0,1])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Average balanced accuracy')
    # plt.title('Specialized vs General -  OOD')
    plt.legend(bbox_to_anchor=(1.5, 0.5),loc='right', ncol=1)
    # plt.show()
    # plt.savefig('figs/medmnist_specialized_vs_general_ood.eps', bbox_inches='tight', format='eps')
    plt.savefig('figs/medmnist_specialized_vs_general_ood.pdf', bbox_inches='tight', format='pdf')
    
    fig = plt.figure(figsize=(5.25, 3), dpi=80)
    # plt.subplot(1,2,2)
    plt.errorbar(sampling_percentage_vec,mean_eval_sec_id_mat[:,0],yerr=std_eval_sec_id_mat[:,0],label='Specialized - ' +'0%',color=blue_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.errorbar(sampling_percentage_vec,mean_eval_red_sec_id_mat[:,i],yerr=std_eval_red_sec_id_mat[:,i],label='Specialized - ' + str(reduced_percentage_vec[i])+'%',color=blue_colors[i+1])
    plt.errorbar(sampling_percentage_vec,mean_eval_id_mat[:,0],yerr=std_eval_id_mat[:,0],label='Multi-domain - ' +'0%',color=red_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.errorbar(sampling_percentage_vec,mean_eval_red_id_mat[:,i],yerr=std_eval_red_id_mat[:,i],label='Multi-domain - ' + str(reduced_percentage_vec[i])+'%',color=red_colors[i+1])
    plt.ylim([0.5,1])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Average balanced accuracy')
    # plt.title('Specialized vs General - ID')
    plt.legend(bbox_to_anchor=(1.5, 0.5),loc='right',ncol=1)
    plt.subplots_adjust(wspace=0.5)
    # plt.show()  
    # plt.savefig('figs/medmnist_specialized_vs_general_id.eps', bbox_inches='tight', format='eps')
    plt.savefig('figs/medmnist_specialized_vs_general_id.pdf', bbox_inches='tight', format='pdf')
    
    # plot
    fig = plt.figure(figsize=(5, 3), dpi=80)
    plt.plot(sampling_percentage_vec,mean_eval_mat-mean_eval_sec_mat,label='0%',color=purple_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.plot(sampling_percentage_vec,mean_eval_red_mat[:,i]-mean_eval_red_sec_mat[:,i],label=str(reduced_percentage_vec[i])+'%',color=purple_colors[i+1])
    plt.ylim([-0.05,0.7])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Balanced accuracy difference')
    # plt.title('MedMNIST -  OOD')
    plt.legend(bbox_to_anchor=(1.3, 0.5),loc='right', ncol=1)
    # plt.savefig('figs/medmnist_specialized_vs_general_diff_ood.eps', bbox_inches='tight', format='eps')
    plt.savefig('figs/medmnist_specialized_vs_general_diff_ood.pdf', bbox_inches='tight', format='pdf')
    
    fig = plt.figure(figsize=(5, 3), dpi=80)
    plt.plot(sampling_percentage_vec,mean_eval_id_mat-mean_eval_sec_id_mat,label='0%',color=purple_colors[0])
    for i in range(0,len(reduced_percentage_vec)):
        plt.plot(sampling_percentage_vec,mean_eval_red_id_mat[:,i]-mean_eval_red_sec_id_mat[:,i],label=str(reduced_percentage_vec[i])+'%',color=purple_colors[i+1])
    plt.ylim([-0.05,0.05])
    plt.xlabel('Sampling percentage [%]')
    plt.ylabel('Balanced accuracy difference')
    # plt.title('MedMNIST - ID')
    plt.legend(bbox_to_anchor=(1.3, 0.5),loc='right',ncol=1)
    plt.subplots_adjust(wspace=0.5)
    # plt.show() 
    # plt.savefig('figs/medmnist_specialized_vs_general_diff_id.eps', bbox_inches='tight', format='eps')
    plt.savefig('figs/medmnist_specialized_vs_general_diff_id.pdf', bbox_inches='tight', format='pdf')

In [ ]:
from sklearn.metrics import auc

reduced_percentage_vec = [0,25,50,75,85,95,100]

# ood
ood_score = np.zeros([len(reduced_percentage_vec),2])
ood_score[0,0] = auc(sampling_percentage_vec,mean_eval_mat[:,0])
ood_score[0,1] = auc(sampling_percentage_vec,mean_eval_sec_mat[:,0])
for i in range(1,len(reduced_percentage_vec)):
    ood_score[i,0] = auc(sampling_percentage_vec,mean_eval_red_mat[:,i-1]) 
    ood_score[i,1] = auc(sampling_percentage_vec,mean_eval_red_sec_mat[:,i-1]) 
fig = plt.figure(figsize=(4.3, 3), dpi=80)
plt.plot(reduced_percentage_vec,ood_score[:,1],color=blue_colors[0],linewidth=2.5,marker='o',label='Specialized')
plt.plot(reduced_percentage_vec,ood_score[:,0],color=red_colors[0],linewidth=2.5,marker='o',label='Multi-domain')
plt.plot(reduced_percentage_vec,100*np.ones((len(reduced_percentage_vec),1)),'--',color='k')
plt.ylim([0,105])
plt.xlabel('OOD level [%]')
plt.ylabel('AUC')
plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
# plt.show() 
# plt.savefig('figs/medmnist_auc_ood.eps', bbox_inches='tight', format='eps')
plt.savefig('figs/medmnist_auc_ood.pdf', bbox_inches='tight', format='pdf')

# id
id_score = np.zeros([len(reduced_percentage_vec),3])
id_score[0,0] = auc(sampling_percentage_vec,mean_eval_id_mat[:,0])
id_score[0,1] = auc(sampling_percentage_vec,mean_eval_sec_id_mat[:,0])
for i in range(1,len(reduced_percentage_vec)):
    id_score[i,0] = auc(sampling_percentage_vec,mean_eval_red_id_mat[:,i-1]) 
    id_score[i,1] = auc(sampling_percentage_vec,mean_eval_red_sec_id_mat[:,i-1])   
fig = plt.figure(figsize=(4.3, 3), dpi=80)
plt.plot(reduced_percentage_vec,id_score[:,1],color=blue_colors[0],linewidth=2.5,marker='o',label='Specialized')
plt.plot(reduced_percentage_vec,id_score[:,0],color=red_colors[0],linewidth=2.5,marker='o',label='Multi-domain')
plt.plot(reduced_percentage_vec,100*np.ones((len(reduced_percentage_vec),1)),'--',color='k')
plt.ylim([0,105])
plt.xlabel('OOD level [%]')
plt.ylabel('AUC')
plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
# plt.show()
# plt.savefig('figs/medmnist_auc_id.eps', bbox_inches='tight', format='eps')
plt.savefig('figs/medmnist_auc_id.pdf', bbox_inches='tight', format='pdf')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import matplotlib.colors as mcolors
from sklearn.metrics import auc

# plot for config['sample_class']
data_dir = data_path + '/Data/MedMNIST/'
results_dir = results_path + '/MedMNIST/results/results_Organ/'
results_dir_red = results_path + '/MedMNIST/results/results_Reduced/'
results_dir_sec = results_path + '/MedMNIST/results/results_Secondary/'
results_dir_red_sec = results_path + '/MedMNIST/results/results_ReducedSecondary/'

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

# hard-code MedMNIST has 3 views and 11 organs
CLASS_NAMES = []
CLASS_NAMES_exp = []
CLASS_NAMES_exp.append(['Bladder','Left femoral head','Right femoral head','Heart','Left kidney',
                        'Right kidney','Liver','Left lung','Right lung','Pancreas','Spleen'])
CLASS_NAMES_exp.append(['Axial','Coronal','Sagittal'])
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[0])))
CLASS_NAMES.append(range(0,len(CLASS_NAMES_exp[1])))
        
# x and y_axis ticks
y_list = CLASS_NAMES_exp[0]  
x_list = CLASS_NAMES_exp[1]  

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

# iterate over seeds
seed_vec = [42,73,666,777,1009]# ,1279,1597,1811,1949,2053]
model_type_vec = ['ResNet18'] # ,'DenseNet121' 
sampling_percentage_vec = [100,75,50,35,25,10,5]

blue_colors = [mcolors.CSS4_COLORS['darkslateblue'],
               mcolors.CSS4_COLORS['midnightblue'],
                mcolors.CSS4_COLORS['darkblue'],
                mcolors.CSS4_COLORS['mediumblue'],
                mcolors.CSS4_COLORS['royalblue'],
                mcolors.CSS4_COLORS['cornflowerblue'],
                mcolors.CSS4_COLORS['lightblue']]

red_colors = [mcolors.CSS4_COLORS['darkred'],
                  mcolors.CSS4_COLORS['firebrick'],
                  mcolors.CSS4_COLORS['crimson'],
                  mcolors.CSS4_COLORS['red'],
                  mcolors.CSS4_COLORS['orangered'],
                  mcolors.CSS4_COLORS['orange'],
                  mcolors.CSS4_COLORS['gold']]

purple_colors = [mcolors.CSS4_COLORS['indigo'],
                mcolors.CSS4_COLORS['purple'],
                mcolors.CSS4_COLORS['darkviolet'],
                mcolors.CSS4_COLORS['mediumorchid'],
                mcolors.CSS4_COLORS['orchid'],
                mcolors.CSS4_COLORS['thistle'],
                mcolors.CSS4_COLORS['lavender']]

# iterate for each view
for model_type in model_type_vec:
    print('Model type: ' + model_type)
    
    for view in range(0,num_classes[1]):
        if view == 0:
            view_str = 'Axial'
        elif view == 1:
            view_str = 'Coronal'
        else:
            view_str = 'Sagittal'
        print('View: ' + view_str)
        
        reduced_percentage_vec = [25,50,75,85,95,100] 

        # initialize
        mean_eval_mat = np.zeros([len(sampling_percentage_vec),1])
        mean_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
        mean_eval_red_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        mean_eval_red_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        std_eval_mat = np.zeros([len(sampling_percentage_vec),1])
        std_eval_id_mat = np.zeros([len(sampling_percentage_vec),1])
        std_eval_red_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        std_eval_red_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        mean_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
        mean_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
        mean_eval_red_sec_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        mean_eval_red_sec_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        std_eval_sec_mat = np.zeros([len(sampling_percentage_vec),1])
        std_eval_sec_id_mat = np.zeros([len(sampling_percentage_vec),1])
        std_eval_red_sec_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        std_eval_red_sec_id_mat = np.zeros([len(sampling_percentage_vec),len(reduced_percentage_vec)])
        cnt_sampling_percentage = 0
    
        # iterate over sampling percentage
        for sampling_percentage in sampling_percentage_vec:
            # print
            print('Sampling percentage: ' + str(sampling_percentage))
    
            # initlaize eval_mat
            eval_mat = np.zeros([num_classes[0],1])
            eval_id_mat = np.zeros([num_classes[0],1])
            eval_vec = np.zeros([len(seed_vec),1])
            eval_id_vec = np.zeros([len(seed_vec),1])
            eval_sec_mat = np.zeros([num_classes[0],1])
            eval_sec_id_mat = np.zeros([num_classes[0],1])
            eval_sec_vec = np.zeros([len(seed_vec),1])
            eval_sec_id_vec = np.zeros([len(seed_vec),1])
            cnt_seed = 0
    
            # iterate over random seed
            for seed in seed_vec:     
        
                # get the current general experiment
                df_ = df[(df['seed'] == seed) & 
                     (df['model_type'] == model_type) &
                     (df['sampling_percentage'] == sampling_percentage)]
                curr_exp = int(df_['experiment_id'])
                curr_path = results_dir + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                if os.path.exists(curr_path):
                    with open(curr_path, 'r') as f:
                        data = json.load(f) 
                
                    eval_ = np.array(data['eval_mat_sec'])
                    eval_ = eval_[:,view]
                    eval_id = np.zeros([num_classes[0],num_classes[1]])
            
                    # calculate id accuracy 
                    for j in range(num_classes[0]):
                            tmp = np.array(data['eval_mat_sec'], copy=True)  
                            tmp = tmp[:,view]
                            tmp[j] = float('nan')
                            eval_id[j] = np.nanmean(tmp) 
                    
                # sum for each seed
                eval_mat = eval_mat + eval_
                eval_id_mat = eval_id_mat + eval_id
                eval_vec[cnt_seed] = np.nanmean(eval_)
                eval_id_vec[cnt_seed] = np.nanmean(eval_id)
    
                # filter df_nbrData_secondary
                eval_sec = np.zeros([num_classes[0],1])
                eval_sec_id = np.zeros([num_classes[0],1])
        
                # get the current experiment
                df_sec_ = df_secondary[(df_secondary['task2'] == view) & 
                                   (df_secondary['seed'] == seed) & 
                                   (df_secondary['model_type'] == model_type) & 
                                   (df_secondary['sampling_percentage'] == sampling_percentage)]
                curr_exp = int(df_sec_['experiment_id'])
                curr_path = results_dir_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                if os.path.exists(curr_path):
                    with open(curr_path, 'r') as f:
                        data = json.load(f) 
                
                    # calculate id accuracy 
                    for j in range(num_classes[0]):
                        eval_sec[j] = np.array(data['eval_mat'])[j]
                        tmp = np.array(data['eval_mat'], copy=True)  
                        tmp[j] = float('nan')
                        eval_sec_id[j] = np.nanmean(tmp) 
                    
                # sum for each seed
                eval_sec_mat = eval_sec_mat + eval_sec
                eval_sec_id_mat = eval_sec_id_mat + eval_sec_id
                eval_sec_vec[cnt_seed] = np.nanmean(eval_sec)
                eval_sec_id_vec[cnt_seed] = np.nanmean(eval_sec_id)
                cnt_seed = cnt_seed + 1
        
            # assign for each sampling_percentage
            # general
            eval_mat = eval_mat/cnt_seed
            eval_id_mat = eval_id_mat/cnt_seed
            mean_eval_mat[cnt_sampling_percentage] = np.nanmean(eval_vec)
            mean_eval_id_mat[cnt_sampling_percentage] = np.nanmean(eval_id_vec)
            std_eval_mat[cnt_sampling_percentage] = np.nanstd(eval_vec)
            std_eval_id_mat[cnt_sampling_percentage] = np.nanstd(eval_id_vec)
            # secondary
            eval_sec_mat = eval_sec_mat/cnt_seed
            eval_sec_id_mat = eval_sec_id_mat/cnt_seed
            mean_eval_sec_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_vec)
            mean_eval_sec_id_mat[cnt_sampling_percentage] = np.nanmean(eval_sec_id_vec)
            std_eval_sec_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_vec)
            std_eval_sec_id_mat[cnt_sampling_percentage] = np.nanstd(eval_sec_id_vec)
    
            # iterate over reduced secondary experiments
            cnt_reduced_percentage = 0
    
            for reduced_percentage in reduced_percentage_vec:
                # initialize
                eval_red_mat = np.zeros([num_classes[0],1])
                eval_red_id_mat = np.zeros([num_classes[0],1])
                eval_red_vec = np.zeros([len(seed_vec),1])
                eval_red_id_vec = np.zeros([len(seed_vec),1])
                eval_red_sec_mat = np.zeros([num_classes[0],1])
                eval_red_sec_id_mat = np.zeros([num_classes[0],1])
                eval_red_sec_vec = np.zeros([len(seed_vec),1])
                eval_red_sec_id_vec = np.zeros([len(seed_vec),1])
                cnt_seed = 0
        
                # iterate over random seed
                for seed in seed_vec: 
            
                    # initialize
                    eval_red = np.zeros([num_classes[0],1])
                    eval_red_id = np.zeros([num_classes[0],1])
            
                    # get the current experiment
                    df_red = df_reduced[(df_reduced['seed'] == seed) & 
                                (df_reduced['model_type'] == model_type) & 
                                (df_reduced['sampling_percentage'] == sampling_percentage) & 
                                (df_reduced['reduced_percentage'] == reduced_percentage)]
            
                    # initialize cnt
                    for j in range(num_classes[0]):
                        # get the current experiment
                        df_red_ = df_red[(df_red['task1'] == j) & (df_red['task2'] == view)]
                        curr_exp = int(df_red_['experiment_id'])
                        curr_path = results_dir_red + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                        if os.path.exists(curr_path):
                            with open(curr_path, 'r') as f:
                                data = json.load(f) 
                
                            eval_red[j] = np.array(data['eval_mat_sec'])[j,view]
                            tmp = np.array(data['eval_mat_sec'], copy=True)  
                            tmp = tmp[:,view]
                            tmp[j] = float('nan')
                            eval_red_id[j] = np.nanmean(tmp) 
                    # sum for each seed
                    eval_red_mat = eval_red_mat + eval_red
                    eval_red_id_mat = eval_red_id_mat + eval_red_id
                    eval_red_vec[cnt_seed] = np.nanmean(eval_red)
                    eval_red_id_vec[cnt_seed] = np.nanmean(eval_red_id)
            
                    # initialize
                    eval_red_sec = np.zeros([num_classes[0],1])
                    eval_red_sec_id = np.zeros([num_classes[0],1])
            
                    # get the current experiment
                    df_red_sec = df_reduced_secondary[(df_reduced_secondary['seed'] == seed) & 
                                             (df_reduced_secondary['model_type'] == model_type) &      
                                             (df_reduced_secondary['sampling_percentage'] == sampling_percentage) & 
                                             (df_reduced_secondary['reduced_percentage'] == reduced_percentage)]
            
                    for j in range(num_classes[0]):
                        # get the current experiment
                        df_red_sec_ = df_red_sec[(df_red_sec['task1'] == j) & (df_red_sec['task2'] == view)]
                        curr_exp = int(df_red_sec_['experiment_id'])
                        curr_path = results_dir_red_sec + 'experiment_' + str(curr_exp) + '/' + str(curr_exp) + '_results.json'
                        if os.path.exists(curr_path):
                            with open(curr_path, 'r') as f:
                                data = json.load(f) 
                
                            eval_red_sec[j] = np.array(data['eval_mat'])[j]
                            tmp = np.array(data['eval_mat'], copy=True)  
                            tmp[j] = float('nan')
                            eval_red_sec_id[j] = np.nanmean(tmp) 
                    
                    # sum for each seed
                    eval_red_sec_mat = eval_red_sec_mat + eval_red_sec
                    eval_red_sec_id_mat = eval_red_sec_id_mat + eval_red_sec_id
                    eval_red_sec_vec[cnt_seed] = np.nanmean(eval_red_sec)
                    eval_red_sec_id_vec[cnt_seed] = np.nanmean(eval_red_sec_id)
                    cnt_seed = cnt_seed + 1   
            
                # assign for each sampling_percentage
                # general
                eval_red_mat = eval_red_mat/cnt_seed
                eval_red_id_mat = eval_red_id_mat/cnt_seed
                mean_eval_red_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_vec)
                mean_eval_red_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_id_vec)
                std_eval_red_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_vec)
                std_eval_red_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_id_vec)
                # secondary
                eval_red_sec_mat = eval_red_sec_mat/cnt_seed
                eval_red_sec_id_mat = eval_red_sec_id_mat/cnt_seed
                mean_eval_red_sec_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_sec_vec)
                mean_eval_red_sec_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanmean(eval_red_sec_id_vec)
                std_eval_red_sec_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_sec_vec)
                std_eval_red_sec_id_mat[cnt_sampling_percentage,cnt_reduced_percentage] = np.nanstd(eval_red_sec_id_vec)
                cnt_reduced_percentage = cnt_reduced_percentage + 1   
    
            # increment cnt for sampling percentage
            cnt_sampling_percentage = cnt_sampling_percentage + 1   
        
        # plot
        fig = plt.figure(figsize=(5.25, 3), dpi=80)
        # plt.subplot(1,2,1)
        plt.errorbar(sampling_percentage_vec,mean_eval_sec_mat[:,0],yerr=std_eval_sec_mat[:,0],label='Specialized - '+ '0%',color=blue_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.errorbar(sampling_percentage_vec,mean_eval_red_sec_mat[:,i],yerr=std_eval_red_sec_mat[:,i],label='Specialized - ' + str(reduced_percentage_vec[i])+'%',color=blue_colors[i+1])
        plt.errorbar(sampling_percentage_vec,mean_eval_mat[:,0],yerr=std_eval_mat[:,0],label='Multi-domain - '+ '0%',color=red_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.errorbar(sampling_percentage_vec,mean_eval_red_mat[:,i],yerr=std_eval_red_mat[:,i],label='Multi-domain - ' + str(reduced_percentage_vec[i])+'%',color=red_colors[i+1])
        plt.ylim([0,1])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Average balanced accuracy')
        # plt.title(view_str + ' - Specialized vs General -  OOD')
        plt.legend(bbox_to_anchor=(1.5, 0.5),loc='right', ncol=1)
        # plt.savefig('figs/medmnist_'+view_str+'_ood.eps', bbox_inches='tight', format='eps')
        plt.savefig('figs/medmnist_'+view_str+'_ood.pdf', bbox_inches='tight', format='pdf')
        
        fig = plt.figure(figsize=(5.25, 3), dpi=80)
        # plt.subplot(1,2,2)
        plt.errorbar(sampling_percentage_vec,mean_eval_sec_id_mat[:,0],yerr=std_eval_sec_id_mat[:,0],label='Specialized - ' +'0%',color=blue_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.errorbar(sampling_percentage_vec,mean_eval_red_sec_id_mat[:,i],yerr=std_eval_red_sec_id_mat[:,i],label='Specialized - ' + str(reduced_percentage_vec[i])+'%',color=blue_colors[i+1])
        plt.errorbar(sampling_percentage_vec,mean_eval_id_mat[:,0],yerr=std_eval_id_mat[:,0],label='Multi-domain - ' +'0%',color=red_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.errorbar(sampling_percentage_vec,mean_eval_red_id_mat[:,i],yerr=std_eval_red_id_mat[:,i],label='Multi-domain - ' + str(reduced_percentage_vec[i])+'%',color=red_colors[i+1])
        plt.ylim([0.5,1])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Average balanced accuracy')
        # plt.title(view_str + ' - Specialized vs General  - ID')
        plt.legend(bbox_to_anchor=(1.5, 0.5),loc='right',ncol=1)
        plt.subplots_adjust(wspace=0.5)
        # plt.show()  
        # plt.savefig('figs/medmnist_'+view_str+'_id.eps', bbox_inches='tight', format='eps')   
        plt.savefig('figs/medmnist_'+view_str+'_id.pdf', bbox_inches='tight', format='pdf')       
    
        # plot
        fig = plt.figure(figsize=(5, 3), dpi=80)
        # plt.subplot(1,2,1)
        plt.plot(sampling_percentage_vec,mean_eval_mat-mean_eval_sec_mat,label='0%',color=purple_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.plot(sampling_percentage_vec,mean_eval_red_mat[:,i]-mean_eval_red_sec_mat[:,i],label=str(reduced_percentage_vec[i])+'%',color=purple_colors[i+1])
        plt.ylim([-0.05,0.7])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Balanced accuracy difference')
        # plt.title(view_str + ' - Difference -  OOD')
        plt.legend(bbox_to_anchor=(1.3, 0.5),loc='right', ncol=1)
        # plt.savefig('figs/medmnist_'+view_str+'_diff_ood.eps', bbox_inches='tight', format='eps')
        plt.savefig('figs/medmnist_'+view_str+'_diff_ood.pdf', bbox_inches='tight', format='pdf')
        
        fig = plt.figure(figsize=(5, 3), dpi=80)
        # plt.subplot(1,2,2)
        plt.plot(sampling_percentage_vec,mean_eval_id_mat-mean_eval_sec_id_mat,label='0%',color=purple_colors[0])
        for i in range(0,len(reduced_percentage_vec)):
            plt.plot(sampling_percentage_vec,mean_eval_red_id_mat[:,i]-mean_eval_red_sec_id_mat[:,i],label=str(reduced_percentage_vec[i])+'%',color=purple_colors[i+1])
        plt.ylim([-0.05,0.05])
        plt.xlabel('Sampling percentage [%]')
        plt.ylabel('Balanced accuracy difference')
        # plt.title(view_str + ' - Difference - ID')
        plt.legend(bbox_to_anchor=(1.3, 0.5),loc='right',ncol=1)
        plt.subplots_adjust(wspace=0.5)
        # plt.show()  
        # plt.savefig('figs/medmnist_'+view_str+'_diff_id.eps', bbox_inches='tight', format='eps')
        plt.savefig('figs/medmnist_'+view_str+'_diff_id.pdf', bbox_inches='tight', format='pdf')

        # auc plot
        reduced_percentage_vec = [0,25,50,75,85,95,100]

        # ood
        ood_score = np.zeros([len(reduced_percentage_vec),2])
        ood_score[0,0] = auc(sampling_percentage_vec,mean_eval_mat[:,0])
        ood_score[0,1] = auc(sampling_percentage_vec,mean_eval_sec_mat[:,0])
        for i in range(1,len(reduced_percentage_vec)):
            ood_score[i,0] = auc(sampling_percentage_vec,mean_eval_red_mat[:,i-1]) 
            ood_score[i,1] = auc(sampling_percentage_vec,mean_eval_red_sec_mat[:,i-1]) 
        fig = plt.figure(figsize=(4.3, 3), dpi=80)
        plt.plot(reduced_percentage_vec,ood_score[:,1],color=blue_colors[0],linewidth=2.5,marker='o',label='Specialized')
        plt.plot(reduced_percentage_vec,ood_score[:,0],color=red_colors[0],linewidth=2.5,marker='o',label='Multi-domain')
        plt.plot(reduced_percentage_vec,100*np.ones((len(reduced_percentage_vec),1)),'--',color='k')
        plt.ylim([0,105])
        plt.xlabel('OOD level [%]')
        plt.ylabel('AUC')
        plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
        # plt.show() 
        # plt.savefig('figs/medmnist_'+view_str+'_auc_ood.eps', bbox_inches='tight', format='eps')
        plt.savefig('figs/medmnist_'+view_str+'_auc_ood.pdf', bbox_inches='tight', format='pdf')

        # id
        id_score = np.zeros([len(reduced_percentage_vec),3])
        id_score[0,0] = auc(sampling_percentage_vec,mean_eval_id_mat[:,0])
        id_score[0,1] = auc(sampling_percentage_vec,mean_eval_sec_id_mat[:,0])
        for i in range(1,len(reduced_percentage_vec)):
            id_score[i,0] = auc(sampling_percentage_vec,mean_eval_red_id_mat[:,i-1]) 
            id_score[i,1] = auc(sampling_percentage_vec,mean_eval_red_sec_id_mat[:,i-1])   
        fig = plt.figure(figsize=(4.3, 3), dpi=80)
        plt.plot(reduced_percentage_vec,id_score[:,1],color=blue_colors[0],linewidth=2.5,marker='o',label='Specialized')
        plt.plot(reduced_percentage_vec,id_score[:,0],color=red_colors[0],linewidth=2.5,marker='o',label='Multi-domain')
        plt.plot(reduced_percentage_vec,100*np.ones((len(reduced_percentage_vec),1)),'--',color='k')
        plt.ylim([0,105])
        plt.xlabel('OOD level [%]')
        plt.ylabel('AUC')
        plt.legend(bbox_to_anchor=(0.5, 1.2),loc='upper center', ncol=2)
        # plt.show()
        # plt.savefig('figs/medmnist_'+view_str+'_auc_id.eps', bbox_inches='tight', format='eps')
        plt.savefig('figs/medmnist_'+view_str+'_auc_id.pdf', bbox_inches='tight', format='pdf')